In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['spark-3.2.1']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.2.1-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()



Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Mod16_Challenge").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Home_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

In [ ]:
# reviews with 20 or more reviews 
twenty_plus_reviews = df.filter(df.total_votes >= 20)
twenty_plus_reviews.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32555853|R27XX9MG7OWSSM|B00CMV99HU|     475264259|5 x 20 Quality Ca...|            Home|          5|           49|         51|   N|                Y|Made Curtains out...|I bought these fo...| 2015-08-31|
|         US|   24278614|R1I4IFDOW6IW9O|B00ZAUWZ6M|     195636450|Ellington Home 18...|            Home|          3|    

In [ ]:
#Helpful reviews dataframe
helpful_reviews = twenty_plus_reviews.filter((twenty_plus_reviews.helpful_votes / twenty_plus_reviews.total_votes) >= 0.5)
helpful_reviews.show(5)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32555853|R27XX9MG7OWSSM|B00CMV99HU|     475264259|5 x 20 Quality Ca...|            Home|          5|           49|         51|   N|                Y|Made Curtains out...|I bought these fo...| 2015-08-31|
|         US|   24278614|R1I4IFDOW6IW9O|B00ZAUWZ6M|     195636450|Ellington Home 18...|            Home|          3|    

In [ ]:
#paid reviews dataframe
paid_reviews = helpful_reviews.filter(helpful_reviews.vine == 'Y')
paid_reviews.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52815715|R2W1OMYJERBXJ1|B00SFUVNKS|     977603375|Panasonic MC-CL94...|            Home|          5|           20|         23|   Y|                N|Compact and powerful|With pets in the ...| 2015-08-31|
|         US|   13663596| R31VJCBBKU13Y|B0105TXVRE|     820889787|Electrolux EL4335...|            Home|          5|    

In [ ]:
#Unpaid reviews dataframe
unpaid_reviews = helpful_reviews.filter(helpful_reviews.vine == 'N')
unpaid_reviews.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32555853|R27XX9MG7OWSSM|B00CMV99HU|     475264259|5 x 20 Quality Ca...|            Home|          5|           49|         51|   N|                Y|Made Curtains out...|I bought these fo...| 2015-08-31|
|         US|   24278614|R1I4IFDOW6IW9O|B00ZAUWZ6M|     195636450|Ellington Home 18...|            Home|          3|    

In [ ]:
# Total # of Paid Reviews
total_paid = paid_reviews.count()
total_paid

1448

In [ ]:
# 5 Star Paid Reviews
paid_five_star= paid_reviews.filter(paid_reviews.star_rating == 5).count()
paid_five_star

647

In [ ]:
# percent of paid 5 star reviews 
paid_five_star_percent = (paid_five_star/total_paid)*100
paid_five_star_percent

44.68232044198895

In [ ]:
#Total Unpaid Reviews 
total_unpaid = unpaid_reviews.count()
total_unpaid

90768

In [ ]:
# total 5 Star unapaid reviews
unpaid_five_star= unpaid_reviews.filter(unpaid_reviews.star_rating == 5).count()
unpaid_five_star

44104

In [ ]:
# Percentage unpaid of 5 star reviews 
unpaid_five_stars_percent = (unpaid_five_star / total_unpaid)*100
unpaid_five_stars_percent

48.589811387273045